In [ ]:
# Base Emotions -
# fear
# self_doubt
# anger
# sadness
# confusion
# comparison
# pressure

In [ ]:
import pandas as pd

df = pd.read_csv("goemotions_1.csv")

# emotions = df["emotion"].unique()
# print(emotions)

print(df.columns)

Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')


In [ ]:

# Normalize go emotions
import pandas as pd

df = pd.read_csv("goemotions_1.csv")
emotion_columns = df.columns[1:]   # All except text
emotion_columns = df.columns.drop("text")

df["emotion"] = df[emotion_columns].apply(
    lambda row: row[row == 1].index[0] if any(row == 1) else "neutral",
    axis=1
)

df_final = df[["text", "emotion"]]

df_final.to_csv("clean_goemotions_1.csv", index=False)

In [ ]:
# Go Emotions after cleaning -
df = pd.read_csv("clean_goemotions_1.csv")

emotions = df["emotion"].unique()
# print(df.columns)
print(emotions)

['rater_id' 'example_very_unclear' 'neutral' 'love' 'gratitude'
 'disapproval' 'amusement' 'disappointment' 'admiration' 'realization'
 'annoyance' 'confusion' 'optimism' 'curiosity' 'excitement' 'caring'
 'disgust' 'remorse' 'joy' 'approval' 'embarrassment' 'surprise' 'anger'
 'grief' 'pride' 'desire' 'sadness' 'relief' 'fear' 'nervousness']


In [ ]:
# Twitter emotions
df = pd.read_csv("data.csv")

emotions = df["Feeling"].unique()
# print(df.columns)
print(emotions)


['happy' 'sad' 'angry' 'fear' 'disgust' 'surprise']


In [ ]:
LABEL_MAP = {
    # fear
    "anxiety": "fear",
    "nervousness": "fear",
    "fear": "fear",

    # sadness
    "sadness": "sadness",
    "grief": "sadness",
    "disappointment": "sadness",

    # anger
    "anger": "anger",
    "annoyance": "anger",
    "frustration": "anger",

    # self doubt
    "shame": "self_doubt",
    "guilt": "self_doubt",
    "embarrassment": "self_doubt",

    # confusion
    "confusion": "confusion",
    "surprise": "confusion",

    # comparison
    "jealousy": "comparison",
    "envy": "comparison",
    "comparison" : "comparison",

    # pressure
    "stress": "pressure",
    "pressure": "pressure"
}


In [ ]:
import pandas as pd

# Load datasets
go = pd.read_csv("clean_goemotions_1.csv")
tw = pd.read_csv("twitter_emotions.csv")
ke = pd.read_csv("emotion_dataset_raw.csv")  # kaggle emotions
st =  pd.read_csv("student_emotion.csv")
# Keep only needed columns
go = go[["text", "emotion"]]
tw = tw[["Tweets", "Feeling"]]
ke = ke[["Text", "Emotion"]]
st = st[["text", "emotion"]]
# Rename for consistency
go.columns = ["text", "raw_emotion"]
tw.columns = ["text", "raw_emotion"]
ke.columns = ["text", "raw_emotion"]
st.columns = ["text", "raw_emotion"]
# Merge datasets
data = pd.concat([go, tw, ke, st], ignore_index=True)


In [ ]:
# Apply mapping
data["emotion"] = data["raw_emotion"].map(LABEL_MAP)

# Drop unmapped rows
data = data.dropna(subset=["emotion"])

# Keep only final columns
data = data[["text", "emotion"]]


In [ ]:
# df = pd.read_csv("emotion_dataset_raw.csv")

# emotions = df["Emotion"].unique()
# print(emotions)

['neutral' 'joy' 'sadness' 'fear' 'surprise' 'anger' 'shame' 'disgust']


In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)   # remove URLs
    text = re.sub(r"[^a-z\s]", "", text)  # keep only letters
    text = re.sub(r"\s+", " ", text)
    return text.strip()

data["text"] = data["text"].apply(clean_text)


In [ ]:
# go = pd.read_csv("clean_goemotions_1.csv")
# emotions = go["emotion"].unique()
# print(emotions)

['rater_id' 'example_very_unclear' 'neutral' 'love' 'gratitude'
 'disapproval' 'amusement' 'disappointment' 'admiration' 'realization'
 'annoyance' 'confusion' 'optimism' 'curiosity' 'excitement' 'caring'
 'disgust' 'remorse' 'joy' 'approval' 'embarrassment' 'surprise' 'anger'
 'grief' 'pride' 'desire' 'sadness' 'relief' 'fear' 'nervousness']


In [ ]:
# Check distribution
print(data["emotion"].value_counts())


emotion
anger         10707
sadness       10332
confusion      7806
fear           7384
comparison     1000
pressure       1000
self_doubt      703
Name: count, dtype: int64


In [ ]:
# balance to a fixed number - 700
FINAL_SAMPLES_PER_CLASS = 700

balanced = (
    data.groupby("emotion", group_keys=False)
        .apply(lambda x: x.sample(
            n=min(len(x), FINAL_SAMPLES_PER_CLASS),
            random_state=42
        ))
)

print(balanced["emotion"].value_counts())

emotion
anger         700
comparison    700
confusion     700
fear          700
pressure      700
sadness       700
self_doubt    700
Name: count, dtype: int64


/tmp/ipython-input-1596060994.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


In [ ]:
final_dataset = balanced.sample(frac=1, random_state=42).reset_index(drop=True)

final_dataset.to_csv("emotion_dataset_final.csv", index=False)
#############         Done with the final Emotion dataset             ############

In [ ]:
### Context Detection dataset ###

pip install google-generativeai

In [ ]:
import google.generativeai as genai
import pandas as pd
import time

genai.configure(api_key="GEMINI_API_KEY")

model = genai.GenerativeModel("gemini-2.5-flash")

labels = [
    "academics",
    "family",
    "peer_pressure",
    "future_career",
    "self_identity"
]

SAMPLES_PER_LABEL = 200

def generate_samples(label, n):
    prompt = f"""
Generate {n} unique student statements.

Context: Academic/student life only.
Label: {label}

Definition:
- academics: Issues with grades, exams, learning, workload.
- family: Conflicts with parents/siblings, household stress.
- peer_pressure: Social bullying, fitting in, forced behaviors.
- future_career: Anxiety about jobs, college, long-term success.
- self_identity: Self-worth, personality, identity struggles.

Rules:
- Each statement must be 15–50 words.
- Natural human tone.
- No numbering.
- No explanations.
- Return ONLY raw sentences.
"""

    response = model.generate_content(prompt)
    text = response.text
    sentences = [s.strip() for s in text.split("\n") if s.strip()]
    return [{"text": s, "context": label} for s in sentences]

all_data = []

for label in labels:
    print(f"Generating {label}")
    data = generate_samples(label, SAMPLES_PER_LABEL)
    all_data.extend(data)
    time.sleep(2)

df = pd.DataFrame(all_data)
df.to_csv("context_dataset.csv", index=False)
print("Saved.")

##### Done with Context Detection Dataset Generation #####